#### 重写dispatch方法
    
    drf重写了django的dispatch方法，新增了以下操作：
    1.对request进行进一步封装；
    2.执行视图函数前，对request依次进行用户认证、权限认证、以及节流限制；（核心步骤）
    3.添加了异常处理（次要）

#### dispatch源码如下

In [1]:
    def dispatch(self, request, *args, **kwargs):
        """
        `.dispatch()` is pretty much the same as Django's regular dispatch,
        but with extra hooks for startup, finalize, and exception handling.
        """
        self.args = args
        self.kwargs = kwargs
        # 对原生的request进行了加工
        # Request(
        #     request,
        #     parsers=self.get_parsers(),
        #     authenticators=self.get_authenticators(),
                #   self.get_authenticators()返回类实例组成的列表
                #   return [auth() for auth in self.authentication_classes]
                    #   authentication_classes = api_settings.DEFAULT_AUTHENTICATION_CLASSES 来自于配置文件

        #     negotiator=self.get_content_negotiator(),
        #     parser_context=parser_context
        # )
        request = self.initialize_request(request, *args, **kwargs)
        self.request = request
        self.headers = self.default_response_headers  # deprecate?

        try:
            # self.initial方法内部实现用户认证、权限认证、节流限制
            self.initial(request, *args, **kwargs)

            # Get the appropriate handler method
            if request.method.lower() in self.http_method_names:
                handler = getattr(self, request.method.lower(),
                                  self.http_method_not_allowed)
            else:
                handler = self.http_method_not_allowed

            response = handler(request, *args, **kwargs)

        except Exception as exc:
            response = self.handle_exception(exc)

        self.response = self.finalize_response(request, response, *args, **kwargs)
        return self.response

#### self.initial()源码

In [ ]:
    def initial(self, request, *args, **kwargs):
        """
        Runs anything that needs to occur prior to calling the method handler.
        """
        self.format_kwarg = self.get_format_suffix(**kwargs)

        # Perform content negotiation and store the accepted info on the request
        neg = self.perform_content_negotiation(request)
        request.accepted_renderer, request.accepted_media_type = neg

        # Determine the API version, if versioning is in use.
        version, scheme = self.determine_version(request, *args, **kwargs)  # 获取版本
        request.version, request.versioning_scheme = version, scheme

        """
            在此依次实现
            用户认证、
            权限认证、
            节流（访问频率）
        """
        self.perform_authentication(request)
        self.check_permissions(request)
        self.check_throttles(request)